# Deep Learning Toolkit for Splunk - Forecasting with Prophet

This notebook contains an example how to use Prophet library for forecasting with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [ ]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
from fbprophet import Prophet
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("Prophet: " + str(Prophet))

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| inputlookup bluetooth.csv</br>
| where probe="AxisBoard-5" </br>
| timechart dc(address) as distinct_addresses span=1h </br>
| eval ds=strftime(_time, "%Y-%m-%d"), y=distinct_addresses </br>
| fit MLTKContainer mode=stage algo=prophet_forecast y from ds into app:prophet_forecast </br>

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [ ]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("prophet_forecast")
print(df)
print(param)

## Stage 2 - create and initialize a model

In [ ]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    #X = df[param['feature_variables']]
    #Y = df[param['target_variables']]
    model = Prophet()
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)

## Stage 3 - fit the model

In [ ]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    fit_range_start = int(param['options']['params']['fit_range_start'].lstrip("\"").rstrip("\""))
    fit_range_end = int(param['options']['params']['fit_range_end'].lstrip("\"").rstrip("\""))
    df_fit = df[fit_range_start:fit_range_end]
    model.fit(df_fit)
    info = {"message": "model trained on range " + str(fit_range_start)+":"+str(fit_range_end) }
    return info

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

## Stage 4 - apply the model

In [ ]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    #future = model.make_future_dataframe(periods=365)
    forecast = model.predict(df)
    changepoints = pd.DataFrame(model.changepoints)
    changepoints['changepoint'] = 1
    result = pd.concat([forecast, changepoints], axis=1)
    return result

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
apply(model,df,param)

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    model = {}
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code